In [1]:
%load_ext autoreload

%autoreload 2

import glob
import os
import sys
from collections import deque
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import yaml
import yamlordereddictloader

from torchvision import models
from yaml import dump
from scheduling import Scheduling
from generator import Generator, get_mem_props
from generator import *
from utils.visualizer import *
from ir.trace import trace
from ddfg_main import synthesis_hardware
from main import design_tech_runner, design_runner, run_single

Unable to import mlperf_logging,  No module named 'mlperf_logging'
Unable to import onnx.  No module named 'onnx'


In [2]:
from common_models import alexnet_graph, vggnet_graph, resnet_graph, bert_graph, gpt2_graph, dlrm_graph, alexnet_graph, langmodel_graph
# design_runner([vggnet_graph()])
# # for node in dlrm_graph.nodes:
# #     print(node.in_edge_mem + node.mem_fetch + node.out_edge_mem, node.compute_expense )

# design_tech_runner([dlrm_graph])
vgg11_graph = vggnet_graph()
resnet_graph_data = resnet_graph()
alexnet_data = alexnet_graph()
langmod_graph1, langmod_graph2 = langmodel_graph()
# dlrm_graph_data = dlrm_graph()
bert_graph_data = bert_graph()
gpt2_graph_data = gpt2_graph()
# 
# from nonai_models.graph_processing import graph_processing

I0204 23:11:21.177439 139634918086400 file_utils.py:39] PyTorch version 1.5.0 available.
/home/khushal/anaconda2/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Example 1 : Resnet 50 : Performance Estimation 

In [3]:
run_single(resnet_graph_data, backprop=False, print_stats=True, filename = 'illusion_nvm.yaml', mapping="nn_dataflow")

Time 1165254 6384 160534
Energy 4989825294 971181747 3762051399 5521181 4730224 9217617 2653334 1200954 9217617868
Area 285631 50462 24499 210669
memory accesses 53073992 53073992 25505960 11544528
rf access 2044593152.0
Design Params 
 No. of PEs :  4096 
 Memory Level-1 Connectivity :  256 
 Memory Level-0 Size :  2000000 
 Memory Level-0 Read Energy :  0.104028
Tech Params [1, 1, 40]


([1165254.541015625, 6384, 160534, 0],
 [4989825294, 971181747, 5521181, 4730224, 9217617, 163238144, 73884979, 0],
 285631.669568)

## Example 2 : Non-AI Graph Processing : Architecture Synthesis and Performance Estimation 

In [4]:
import ast

from ir.staticfg.staticfg import CFGBuilder
cfg = CFGBuilder().build_from_file(
    "aes.py",
    "/media/khushal/679f068d-921b-4d14-890f-3081c1728f98/research/SM/dragon-project/src/nonai_models/aes.py",
)
cfg.build_visual("aes", "pdf", show=True)
print(cfg)

Subscript(value=Name(id='matrix', ctx=Load()), slice=Index(value=BinOp(left=Name(id='i', ctx=Load()), op=Div(), right=Num(n=4))), ctx=Load())
Subscript(value=Attribute(value=Name(id='self', ctx=Load()), attr='round_keys', ctx=Load()), slice=Index(value=Name(id='i', ctx=Load())), ctx=Load())
Subscript(value=Attribute(value=Name(id='self', ctx=Load()), attr='round_keys', ctx=Load()), slice=Index(value=Name(id='i', ctx=Load())), ctx=Load())
Subscript(value=Attribute(value=Name(id='self', ctx=Load()), attr='round_keys', ctx=Load()), slice=Index(value=Name(id='i', ctx=Load())), ctx=Load())
CFG for aes.py


In [23]:
synthesis_hardware("bfs")

-------------------------------
bfs_bulk,machsuite/bfs_bulk/inputs/dynamic_trace1.gz,machsuite/bfs_bulk/test_bfs.cfg,
-------------------------------
      Setting ScratchPad       
-------------------------------
-------------------------------
      Generating DDDG          
-------------------------------
-------------------------------1
-------------------------------2
-------------------------------
Num of Nodes: 62791
Num of Edges: 135460
Num of Reg Edges: 68139
Num of MEM Edges: 4531
Num of Control Edges: 62790
-------------------------------
-------------------------------
    Initializing BaseDatapath      
-------------------------------
 Top level: bfs
      Optimizing...            bfs_bulk
  Removing 45453 edges.
  Adding 37644 new edges.
  Removing 10058 isolated nodes.
-------------------------------
        Mem to Reg Conv        
-------------------------------
-------------------------------
      ScratchPad Partition     
-------------------------------
cache_size 40

In [24]:
synthesis_hardware("aes")

-------------------------------
aes,machsuite/aes_aes/inputs/dynamic_trace.gz,machsuite/aes_aes/test_aes.cfg,
-------------------------------
      Setting ScratchPad       
-------------------------------
-------------------------------
      Generating DDDG          
-------------------------------
-------------------------------1
-------------------------------2
-------------------------------
Num of Nodes: 14811
Num of Edges: 35693
Num of Reg Edges: 19191
Num of MEM Edges: 2132
Num of Control Edges: 14810
-------------------------------
-------------------------------
    Initializing BaseDatapath      
-------------------------------
 Top level: aes256_encrypt_ecb
      Optimizing...            aes
  Removing 9081 edges.
  Adding 6506 new edges.
  Removing 491 isolated nodes.
-------------------------------
        Mem to Reg Conv        
-------------------------------
-------------------------------
      ScratchPad Partition     
-------------------------------
cache_size 256
l

## Architecture Generation  : Google Bert

In [11]:
backprop = False
import time
start = time.time()
# name = ["vgg_graph", "resnet_graph","bert_graph","gpt2_graph", "dlrm_graph"]
# graph_list = [vgg11_graph, resnet_graph_data, bert_graph_data, gpt2_graph_data, dlrm_graph_data]
graph_list = [bert_graph_data]
name = ["bert_graph_data"]

for i,graph in enumerate(graph_list):
# for graph in [bert_graph_data]:
    log_file_name = "logs/"+str(name[i])+str(backprop)
    arch_time, arch_energy, arch_area = design_runner([graph], backprop, print_stats= True, stats_file = log_file_name)
# end = time.time()
# print(end-start)

Time 7054999 3106472 2615926
Energy 161671287 24098673 78463547 13178116 10550143 6988182 6526513 117657 6988182094
Area 158827 50462 382 107982
memory accesses 176338576 176338576 87332368 1574400
rf access 42643232.499999896
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  2 
 Memory Level-0 Size :  1048576 
 Memory Level-0 Read Energy :  0.07473189999999999
Tech Params [1, 1, 40]
======Optimizing Design=========
Time 7054999 3106472 2615926
Energy 161671287 24098673 78463547 13178116 10550143 6988182 6526513 117657 6988182094
Area 158827 50462 382 107982
memory accesses 176338576 176338576 87332368 1574400
rf access 42643232.499999896
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  2 
 Memory Level-0 Size :  1048576 
 Memory Level-0 Read Energy :  0.07473189999999999
Tech Params [1, 1, 40]
Time 3605617 146908 2126079
Energy 154803837 24099197 78465255 13174896 10547565 3571468 6526513 118346 3571468663
Area 158827 50462 382 107982
memory accesses 176

Time 2795824 143596 1319591
Energy 165138634 24099324 78465667 17761731 15146863 5535285 8840655 157821 5535285376
Area 264366 50462 382 213521
memory accesses 175459168 175459168 87332368 1559040
rf access 42644384.50000003
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  6 
 Memory Level-0 Size :  1728289 
 Memory Level-0 Read Energy :  0.10123
Tech Params [1, 1, 40]
Time 3454406 146246 1975505
Energy 167127901 24099649 78466727 17777039 15159917 6839172 8840655 156577 6839172098
Area 264366 50462 382 213521
memory accesses 175610384 175610384 87332368 1546752
rf access 42644960.500000045
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  6 
 Memory Level-0 Size :  1742448 
 Memory Level-0 Read Energy :  0.10123
Tech Params [1, 1, 40]
Time 7647521 147531 6167328
Energy 179636888 24099776 78467139 17785990 15167550 15140869 8840655 155333 15140869560
Area 264366 50462 382 213521
memory accesses 175698808 175698808 87332368 1534464
rf access 42645184.49999

Time 6779822 146287 5300900
Energy 176444536 24099288 78465549 17469581 14897723 13422963 8840655 147559 13422963217
Area 264366 50462 382 213521
memory accesses 172573169 172573169 87332368 1457664
rf access 42644320.50000006
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  6 
 Memory Level-0 Size :  2451099 
 Memory Level-0 Read Energy :  0.10123
Tech Params [1, 1, 40]
Time 1455539 122951 0
Energy 144579487 24098438 78462782 9024834 7696205 2881735 8840655 77 2881735257
Area 264366 50462 382 213521
memory accesses 89151776 89151776 87332368 768
rf access 42642816.49999997
Design Params 
 No. of PEs :  1 
 Memory Level-1 Connectivity :  6 
 Memory Level-0 Size :  2474554 
 Memory Level-0 Read Energy :  0.10123
Tech Params [1, 1, 40]
4.847000140887055 1.1182173235958432 0.6007853213621843


## Technology Targets Generation